How to start a webdriver client in node?

In [1]:
var execSync = require('child_process').execSync;
try {
    require.resolve('webdriverio');
    require.resolve('wdio-sync');
} catch (e) {
    execSync('npm install webdriverio wdio-sync');
}
var webdriverio = require('webdriverio');
var sync = require('wdio-sync')

var createWebdriverClient = (host, port) => {
    var webdriverServer = {
        sync: true,
        debug: true,
        host: host || 'localhost',
        port: port || 4444,
        pageLoadStrategy: 'eager',
        connectionRetryTimeout: 10000,
        desiredCapabilities: {
            browserName: 'chrome',
            chromeOptions: {
                prefs: {
                    'download.default_directory' : '/data/downloads'
                },
                args: [
                    'user-data-dir=/usr/profile', 
                    'start-fullscreen', 
                    'no-sandbox',
                    //'disable-session-crashed-bubble',
                    //'disable-infobars',
                    //'kiosk'
                ]
            }
        }
    };
    
    // TODO: add decorated logging with screenshots of buttons 
    // results can be used: https://github.com/megamindbrian/bots/blob/master/bots/server.js
    
    // TODO: transfer state and cache to client

    console.log('Initializing webdriver on ' + webdriverServer.host);
    var client = webdriverio.remote(webdriverServer);
    sync.wrapCommands(client, [], [])
    client.$ = (...config) => client.element.apply(client, config)
    client.$$ = (...config) => client.elements.apply(client, config).value
    client.on('error', function (e) {
        console.log(e);
        this.endAll();
    });
    client.on('end', function () {
        console.log('Daemon: Closing browser');
    });
    client.on('result', function (result) {
        //console.log(result);
    });
    return client.sessions()
        .then(r => {
            if(r.value.length == 0) {
                return client.init();
            } else {
                client.requestHandler.sessionID = r.value[0].id;
            }
            return client.windowHandles();
        })
        .then(r => {
            if(r.value.length == 0) {
                return client.newWindow();
            }
        });
};
module.exports = createWebdriverClient;
createWebdriverClient;



[Function: createWebdriverClient]

In [2]:
$$.async();
var client = createWebdriverClient('localhost', 4444)
    .then(() => $$.sendResult(client.requestHandler.sessionID))
    .catch(e => $$.sendError(e));


Initializing webdriver on localhost


'31094014-8005-453e-951b-100b66185c1f'

In [5]:
$$.async();
client.windowHandles()
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e));

TypeError: Cannot read property 'windowHandles' of undefined

How to end the webdriver service?

In [ ]:
client.endAll();
